In [3]:
import numpy as np
import math

## Utils

In [2]:
from sample import _generate_puzzle
import utils


def generate_puzzle(dimension, alpha, error_rate=0):
    w = utils.random_unit_vector(dimension)
    a, M1, _ = _generate_puzzle(w, dimension, alpha, error_rate=0)
    b, M2, coserror = _generate_puzzle(w, dimension, alpha, error_rate=error_rate)
    return a, b, M2 @ M1.T, coserror

## Overall Settings

In [3]:
# parameter of codewords
dimension = 512
alpha = 16

## Parameter Explanation and Example

### parameter explanation

Settings

In [8]:
# whether testing real angle with desired_success_rate
desired_success_rate_test = True
desired_success_rate = 0.8

parameter explanation

In [9]:
print("dimension  = %d, alpha(parameter of codeword) = %d" % (dimension, alpha))
print("Log(size of codewords) = %d" % (utils.log2c_alpha(dimension, alpha) + alpha))
print(
    "Design (error-torelant) angle = {:.2f}".format(
        np.arccos(1 - 1 / alpha) / np.pi * 180 / 2
    )
)

if desired_success_rate_test:
    print(
        "Real (error-torelant) angle = {:.2f} degree with success rate {:.1f}%".format(
            utils.get_real_error_torelant_angle(dimension, alpha, desired_success_rate),
            desired_success_rate * 100,
        )
    )
print("*****************************************************************")
print(
    "*IronMask parameters settings are:\n*\tdimension = %d, alpha = %d\n"
    "*you could change the parameter to align to IronMask's settings but the time would be long\n"
    "*If you just to verify that the code is runnable, then recommended parameters are:\n"
    "*\tdimension = %d, alpha = %d" % (512, 16, 512, 4)
)

dimension  = 512, alpha(parameter of codeword) = 16
Log(size of codewords) = 115
Design (error-torelant) angle = 10.18


Testing real error torelant angle:   0%|          | 0/90 [00:00<?, ?it/s]

Real (error-torelant) angle = 45.00 degree with success rate 80.0%
*****************************************************************
*IronMask parameters settings are:
*	dimension = 512, alpha = 16
*you could change the parameter to align to IronMask's settings but the time would be long
*If you just to verify that the code is runnable, then recommended parameters are:
*	dimension = 512, alpha = 4


### Example

Setting

In [10]:
error_angle = 40  # degree

Example 1: two sketch

In [11]:
from ironmask import decode_codeword

error_rate = utils.translate_error_angle_to_error_noise(error_angle)
a, b, isometric_matrix, coserror = generate_puzzle(
    dimension, alpha, error_rate=error_rate
)
print("Judgements: ")
# check if it is isometric
print(
    "* Is the puzzle isometric? ",
    np.allclose(np.eye(dimension), np.dot(isometric_matrix, isometric_matrix.T)),
)
# check if it maps a to b
decoded_codeword = decode_codeword(isometric_matrix @ a, dimension, alpha)
print(
    "* Does the puzzle could be decoded from template a to codeword b? ",
    np.allclose(b, decoded_codeword, atol=1e-3),
    "\n    Non-zero elements indices of each vector:\n    - Decode(M @ a):\t",
    np.sort(np.argsort(np.abs(decoded_codeword))[-alpha:]),
    "\n    -        b     :\t",
    np.sort(np.argsort(np.abs(b))[-alpha:]),
)
print(
    "* Does the noise error angle is equal to error_angle = %d ? " % error_angle,
    np.allclose(utils.get_angle_of_two_vectors(isometric_matrix @ a, b), error_angle),
)

Judgements: 
* Is the puzzle isometric?  True
* Does the puzzle could be decoded from template a to codeword b?  True 
    Non-zero elements indices of each vector:
    - Decode(M @ a):	 [ 14  31  38  76  87 206 210 234 267 316 321 324 368 398 402 473] 
    -        b     :	 [ 14  31  38  76  87 206 210 234 267 316 321 324 368 398 402 473]
* Does the noise error angle is equal to error_angle = 40 ?  True


Example 2: multiple sketches

In [12]:
from sample import generate_puzzle_n
import itertools

error_rate = utils.translate_error_angle_to_error_noise_n(error_angle)
_, cs, isometric_matrices, _ = generate_puzzle_n(
    dimension, alpha, error_rate=error_rate
)
is_isometric = True
for m in isometric_matrices:
    if not np.allclose(np.eye(dimension), np.dot(m, m.T)):
        is_isometric = False
        break
a = isometric_matrices[0].T @ cs[0]
b = cs[1]
isometric_matrix = isometric_matrices[1]
print("Judgements: ")
# check if it is isometric
print(
    "* Is the sketches all isometric? ",
    is_isometric,
)
# check if it maps a to b
decoded_codeword = decode_codeword(isometric_matrix @ a, dimension, alpha)
print(
    "* Does the puzzle map the first noisy template to codeword b(associated with the second noisy template)? ",
    np.allclose(b, decoded_codeword, atol=1e-3),
    "\n    Non-zero elements indices of each vector:\n    - Decode(M @ a):\t",
    np.sort(np.argsort(np.abs(decoded_codeword))[-alpha:]),
    "\n    -        b     :\t",
    np.sort(np.argsort(np.abs(b))[-alpha:]),
)
ws = [isometric_matrices[i].T @ cs[i] for i in range(len(isometric_matrices))]
angles = [
    utils.get_angle_of_two_vectors(ws[i], ws[j], type="degree")
    for i, j in itertools.combinations(range(len(isometric_matrices)), 2)
]

print(
    "* Does the noise error angle is approximately equal to error_angle = %d ? "
    % error_angle,
    np.allclose(np.mean(angles), error_angle, rtol=0.5),
    "\t mean real angle between two noisy templates: {:.2f}".format(np.mean(angles)),
)

Generating Puzzles:   0%|          | 0/511 [00:00<?, ?it/s]

Judgements: 
* Is the sketches all isometric?  True
* Does the puzzle map the first noisy template to codeword b(associated with the second noisy template)?  True 
    Non-zero elements indices of each vector:
    - Decode(M @ a):	 [  8  46  75  84 114 117 122 152 165 195 231 297 351 374 460 466] 
    -        b     :	 [  8  46  75  84 114 117 122 152 165 195 231 297 351 374 460 466]
* Does the noise error angle is approximately equal to error_angle = 40 ?  True 	 mean real angle between two noisy templates: 39.99


## Test for svd getting 2 sketches if sampled matrix is "correct"

### Env

In [13]:
import numpy as np
import utils
from tqdm.auto import tqdm
from attacker import submatrix_solver_numpy
from ironmask import decode_codeword

### Settings

In [14]:
k = 511 # the number of sampled linear equations (k, need to justify in paper section 5 according to noise)
# degree
noise_angle = 0    # Noise(\theta')
threshold = 10  # threshold hyper-parameter(\theta_t, need to justify in paper section 5 according to noise, but generally 40 is enough)

In [15]:
print("Parameters:")
print("The number of sampled linear equations(k):", k)
print("The number of sampled linear equations each sketch:", k // 2)
print("The noise level(Noise(\\theta')):", noise_angle, "degree")
print("The threshold(\\theta_t):", threshold, "degree")

Parameters:
The number of sampled linear equations(k): 511
The number of sampled linear equations each sketch: 255
The noise level(Noise(\theta')): 0 degree
The threshold(\theta_t): 10 degree


### Test code

test success rate

In [16]:
tmpka = k // 2
tmpkb = (k + 1) // 2
error_rate = utils.translate_error_angle_to_error_noise(noise_angle)

In [17]:
num_times = 0
success_times = 0
false_positive_times = 0
for j in (pbar_outer := tqdm(range(10000))):
    # generate a random vector with dimension, which has only k non-zero elements(fill with 1 or -1) in random positions
    a, b, isometric_matrix, coserror = generate_puzzle(
        dimension, alpha, error_rate=error_rate
    )
    near_0_positions_a = np.where(np.abs(a) < 0.1)[0]
    near_0_positions_b = np.argsort(np.abs(b))[: dimension - alpha]
    # assert len(near_0_positions_a) == len(near_0_positions_b)
    # random select dimension // 2 near_0_positions
    test_number = 0
    coserror = 0
    min_angle = 90
    random_near_0_positions_a = np.random.choice(
        near_0_positions_a, tmpka, replace=False
    )
    random_near_0_positions_b = np.random.choice(
        near_0_positions_b, tmpkb, replace=False
    )
    random_near_0_positions_a = np.sort(random_near_0_positions_a)
    random_near_0_positions_b = np.sort(random_near_0_positions_b)
    # get the submatrix of isometric_matrix with selected rows: random_near_0_positions_b and columns: versus random_near_0_positions_a
    versus_random_near_0_positions_a = np.setdiff1d(
        np.arange(dimension), random_near_0_positions_a
    )
    submatrix = isometric_matrix[random_near_0_positions_b][
        :, versus_random_near_0_positions_a
    ]
    # get the null vector of submatrix
    null_vector = submatrix_solver_numpy(submatrix)
    null_vector = null_vector / np.linalg.norm(null_vector)
    assume_vector = np.zeros(dimension)
    for i in np.argsort(np.abs(null_vector)):
        assume_vector[versus_random_near_0_positions_a[i]] = null_vector[i]

    # determinant work
    assume_b = np.dot(isometric_matrix, assume_vector)
    decode_b = decode_codeword(assume_b, dimension, alpha)
    # get the arc cos of assume_b and b
    assume_a = np.dot(isometric_matrix.T, assume_b)
    decode_a = decode_codeword(assume_a, dimension, alpha)
    min_angle = utils.get_angle_of_two_vectors(assume_a, decode_a)
    if min_angle < threshold:  # pass determinant check
        if not np.allclose(
            np.sort(np.argsort(np.abs(decode_b))[-alpha:][::-1]),
            np.sort(np.argsort(np.abs(b))[-alpha:]),
        ):
            false_positive_times += 1
        else:
            success_times += 1
    pbar_outer.set_description(
        "success_time: {}, false_positive_times: {}, whole_times: {}".format(
            success_times, false_positive_times, j + 1
        )
    )
    if success_times > 100:
        num_times = j + 1
        break

pkf = success_times / num_times  # p_k * p_f

  0%|          | 0/10000 [00:00<?, ?it/s]

In [21]:
print("P_k * P_f  = {}".format(pkf))

P_k * P_f  = 1.0


Test solve svd time

In [22]:
import numpy as np
import attacker
import time

matrix_size = tmpka
matrix_size_2 = dimension - tmpkb
tmp_num_times = 100
whole_time = 0
for i in range(tmp_num_times):
    tmp_matrix = np.random.rand(matrix_size, matrix_size_2)
    tmp_matrix = tmp_matrix[:-1]
    start_time = time.time()
    attacker.submatrix_solver_numpy(tmp_matrix)
    whole_time += time.time() - start_time

tk = whole_time / tmp_num_times

In [23]:
print("Time(t_k) = {} miliseconds".format(tk * 1000))
print(
    "* Remarks: You might find the time is longer than in the paper, it's because our experiments use intel-mkl to accelerate the svd computation, \n"
    "* while the default svd implementation is openblas, which is 10 times slower than mkl."
)

Time(t_k) = 20.72948932647705 miliseconds
* Remarks: You might find the time is longer than in the paper, it's because our experiments use intel-mkl to accelerate the svd computation, 
* while the default svd implementation is openblas, which is 10 times slower than mkl.


### Results

In [24]:
rk = utils.subset_n_a_alpha(dimension, tmpka, alpha) + utils.subset_n_a_alpha(
    dimension, tmpkb, alpha
)
et = tk / pkf * (2**rk)
print("Whole Time:")
print("* seconds: {:.2f}".format(et))
print("* minutes: {:.2f}".format(et / 60))
print("* hours: {:.2f}".format(et / 3600))
print("* days: {:.2f}".format(et / 3600 / 24))
print("* years: {:.2f}".format(et / 3600 / 24 / 365))

Whole Time:
* seconds: 135393281.88
* minutes: 2256554.70
* hours: 37609.24
* days: 1567.05
* years: 4.29


## Test for svd getting multiple matrices if sampled matrix is "correct"

### Env

In [25]:
# reimport attacker
import importlib
import attacker
import numpy as np
from attacker import solve_puzzle_with_n_matrix_known_places
from sample import generate_puzzle_n
from tqdm.auto import tqdm
importlib.reload(attacker)

<module 'attacker' from 'd:\\code\\probabilistic-linear-regression-attack\\attacker.py'>

### Settings

In [26]:
k = 531  # the number of sampled linear equations (k, need to justify in paper section 5 according to noise), for multiple scenarios, the number of sketches is equal to k
# degree
noise_angle = 8.7  # Noise(\theta')
threshold = 30  # threshold hyper-parameter(\theta_t, need to justify in paper section 5 according to noise, but generally 40 is enough)

In [27]:
print("Parameters:")
print("The number of sampled linear equations(k):", k)
print("The number of sampled linear equations each sketch:", 1)
print("The number of secure sketches:", k)
print("The noise level(Noise(\\theta')):", noise_angle, "degree")
print("The threshold(\\theta_t):", threshold, "degree")

Parameters:
The number of sampled linear equations(k): 531
The number of sampled linear equations each sketch: 1
The number of secure sketches: 531
The noise level(Noise(\theta')): 8.7 degree
The threshold(\theta_t): 30 degree


### Test Code

In [28]:
error_rate = utils.translate_error_angle_to_error_noise_n(noise_angle)
c, bs, isometrixes, coserrors = generate_puzzle_n(dimension, alpha, error_rate = error_rate, n= 2 * dimension, disable_tqdm=False)   # we will select puzzle from this

Generating Puzzles:   0%|          | 0/1024 [00:00<?, ?it/s]

In [29]:
total_run_times = 0
success_times = 0
false_positive_times = 0
total_test_times = 10000
tmpk = k
for j in (pbar := tqdm(range(total_test_times), desc="Test for multiple matrices")):
    # random get tmpk bs and isometrixes
    indices = np.random.choice(len(bs), tmpk, replace=False).tolist()
    tmp_bs = [bs[i] for i in indices]
    tmp_isometrixes = [isometrixes[i] for i in indices]
    result = solve_puzzle_with_n_matrix_known_places(
        tmp_isometrixes,
        tmp_bs,
        dimension,
        alpha,
        max_rtimes=1,
        threshold=threshold,
        disable_tqdm=True,
        return_runtimes=True,
    )
    if result[0] is not None:
        if np.allclose(result[1], tmp_bs[0]) or np.allclose(result[1], -tmp_bs[0]):
            success_times += 1
        else:
            false_positive_times += 1
    total_run_times += result[2]
    pbar.set_postfix(
        {
            "success_times": success_times,
            "averg_success_prob": success_times / total_run_times,
            "false_positive_times": false_positive_times,
        }
    )
    if success_times > 100:
        num_times = j + 1
        break

Test for multiple matrices:   0%|          | 0/10000 [00:00<?, ?it/s]

In [31]:
pkf = success_times / num_times
pf = success_times / (success_times + false_positive_times)
print("P_k * P_f  = {}".format(pkf))
print("P_f = {}", pf)

P_k * P_f  = 0.7062937062937062
P_f = {} 0.9017857142857143


Test solve svd time

In [32]:
import numpy as np
import attacker
import time

matrix_size = k
matrix_size_2 = dimension
tmp_num_times = 100
whole_time = 0
for i in range(tmp_num_times):
    tmp_matrix = np.random.rand(matrix_size, matrix_size_2)
    tmp_matrix = tmp_matrix[:-1]
    start_time = time.time()
    attacker.submatrix_solver_numpy(tmp_matrix)
    whole_time += time.time() - start_time

tk = whole_time / tmp_num_times

In [ ]:
print("Time(t_k) = {} miliseconds".format(tk * 1000))
print(
    "* Remarks: You might find the time is longer than in the paper, it's because our experiments use intel-mkl to accelerate the svd computation, \n"
    "* while the default svd implementation is openblas in Windows Platform, which is 10 times slower than mkl."
)

In [ ]:
# output the implementation backend of numpy BLAS' configuration, you might see OpenBlas instead of MKL
print(np.__config__.show())

### Results

In [34]:
rk = utils.subset_n_a_alpha(dimension, 1, alpha) * k
et = tk / pkf * (2**rk)
print("Whole Time:")
print("* seconds: {:.2f}".format(et))
print("* minutes: {:.2f}".format(et / 60))
print("* hours: {:.2f}".format(et / 3600))
print("* days: {:.2f}".format(et / 3600 / 24))
print("* years: {:.2f}".format(et / 3600 / 24 / 365))

Whole Time:
* seconds: 1990576.76
* minutes: 33176.28
* hours: 552.94
* days: 23.04
* years: 0.06
